In [1]:
#dependencies
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from bayes_opt import BayesianOptimization

In [2]:
#Parameter values

#from Farquhar 1980
#turnover rate for carboxylation
kc = 2.5
#turnover rate for oxygenation
ko = .21*kc
#light intensity
I = 150
#total enzyme concentration
Et = 87.2
#CO2 concentration
C = 230
#O2 concentration
O = 210

#from Ye 2020
alpha = .295
beta = 2.42*(10**(-3))
gamma = 1.26*(10**(-4))

#couldnt find turnover # for RuBP so am setting kr, etc. = kc
kr = kc
kprod = kc
kE = kc

#also guesses here
kERC = .9*kc
kERO = .9*ko

#also note that kappa r and kappa c are 1 for simplicity's sake, and enzyme velocities for PGA reduction and RuBP regeneration are the same

In [3]:
#model for PGA reduction vs RuBP *with Ye model for light
def jprod(R, C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    return jprod

In [4]:
#plotting jprod against RuBP
@interact(Rsteps= (0, 5000, 10), Et = (0, 1000, 10), kr  = (.1, 10, .1))
def simulateR(C = 230, O =210, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = 10, kE = kE, kERO=kERO, kERC = kERC, Rsteps=300):
    vcr = []
    for k in range(1,Rsteps):
        vcr.append(jprod(k, C=C, O=O, kc = kc, ko = ko, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC))
    ksum = ko*O/kERO + kc*C/kERC
    
    plt.plot(range(1,Rsteps), vcr)
    plt.xlabel('RuBP')
    plt.ylabel('jprod')
    plt.show()

interactive(children=(IntSlider(value=230, description='C', max=690, min=-230), IntSlider(value=210, descripti…

In [5]:
#various models for irradiance
#parameters
zun = .21
zdo = .33

In [6]:
#Ye model
def Ye(I, alpha = alpha, beta = beta, gamma = gamma):
    jph = alpha*(1-beta*I)*I/(1+gamma*I)
    return jph

In [7]:
#Ye model with irradiance dependance
def altYe(I, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma):
    frac = beta*I/(zun*I+zdo)
    jph = I*alpha*(1-frac)/(1+gamma*I)
    return jph

In [8]:
#circuit formulation
Imax = 700
Rc = .01
def circuit(I, Imax=Imax, Rc=Rc):
    jph = I/((I/Imax)+Rc)
    return jph

In [9]:
#Various models as functions of irradiance
@interact()
def simulatejph(irange=5000, zun=zun, zdo=zdo, Imax = Imax, Rc=Rc, alpha = alpha, beta = beta, gamma = gamma):
    jphylist = []
    jphalist = []
    jphblist = []
    for i in range(1,irange):
        jphylist.append(Ye(i, alpha = alpha, beta = beta, gamma = gamma))
        jphalist.append(altYe(i, alpha = alpha, beta = beta, zun = zun, zdo = zdo, gamma = gamma))
        jphblist.append(circuit(i, Imax, Rc))
    plt.plot(range(1,irange), jphylist, label='Ye model')
    plt.plot(range(1, irange), jphalist, label = 'model A')
    plt.plot(range(1,irange), jphblist, label = 'model B')
    plt.xlabel('Irradiance')
    plt.ylabel('jph')
    plt.legend()
    plt.show()

interactive(children=(IntSlider(value=5000, description='irange', max=15000, min=-5000), FloatSlider(value=0.2…

In [11]:
#RuBP regeneration
alphaa = 1
alphab = 1
def dRdtalt(R, C = 230, O =210, kc = kc, ko = ko, alphaa = alphaa, alphab = alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    jph = I*alpha*(1-beta*I)/(1+gamma*I)
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    kEa = alphaa*kc
    kEb = alphab*kc
    jpc = (2/3)*jph
    jpr = (1/3)*jph
    jprod = (5/6)*((1/kEa)+(1/jpc)+(1/jpga)-(1/(jpga+jpc)))**(-1)
    jrubp = (1/kEb + 1/jpr + 1/jprod - 1/(jpr+jprod))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + jrubp
    return dRdt

In [24]:
@interact(alpha = (0,100,.1))
#note that R does not scale to realistic values, likely due to wrong parameterization
def simulatealtR(Ro = .0014, tsteps = 500, dt = .001, C = 230, O =210, kc = kc, ko = ko, alphaa = alphaa, alphab=alphab, I = I, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    R = np.zeros(tsteps)
    R[0] = Ro
    for t in range(tsteps-1):
        R[t+1] = R[t] + dRdtalt(R[t], C=C, alpha = alpha, alphaa = alphaa, alphab = alphab)*dt
    plt.plot(range(tsteps),R, label = 'RuBP')
    plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(FloatSlider(value=0.0014, description='Ro', max=0.0042, min=-0.0014), IntSlider(value=50…

In [25]:
#Alternative light model: dynamical system with balancing fluxes
def jprodalt(pq, I, R):
    jph = pq
    ksum = ko*O/kERO + kc*C/kERC
    fE = (1 + ((1/(kr*R))+1)*ksum + kr*R*(1+ksum)/(ko*O + kc*C))**(-1)
    jcA = kc*C*Et*(1+(kr*R/ksum))*fE
    joA = ko*O*Et*(1+(kr*R/ksum))*fE
    jpga = (2*jcA+1.5*joA)
    jprod = ((1/kE)+(1/jph)+(1/jpga)-(1/(jpga+jph)))**(-1)
    dRdt = -1*Et*fE*(kr*R+kc*C+ko*O) + (5/6)*jprod
    return jprod, dRdt

In [26]:
def jpq(pq, I, R, jf=.1*I, fnpq=.2, jsyn=1.5):
    jp, jr = jprodalt(pq, I, R)
    jadp = 2*jp/3 + jr/3
    jatp = (1/jsyn + 1/pq + 1/jadp - 1/(pq + jadp))**(-1)
    jnpq = fnpq*(pq - jatp)
    jpq = I - jf - jnpq
    return jpq, jr, jnpq

In [27]:
#light input
def logistic(Io, Imax, x, kl):
    f = Imax/(1+np.exp(-kl*(x-Io)))
    return f

In [28]:
@interact(pqo = (0,10,.1), Ro=(0,500,10), time = (0,1000,1))
def integrated(pqo, Ro, Io=100, Imax=500, kl = 1, time=1000, dt = .001, C = 230, O =210, kc = kc, ko = ko, Et = Et, alpha = alpha, beta = beta, gamma = gamma, kr = kr, kE = kE, kERO=kERO, kERC = kERC):
    tsteps = np.linspace(0, time, time*10)
    jpqlist = np.zeros(len(tsteps))
    Rlist = np.zeros(len(tsteps))
    npqlist = np.zeros(len(tsteps))
    jpqlist[0] = pqo
    Rlist[0] = Ro
    ilist = [logistic(Io, Imax, t, kl) for t in tsteps]
    for t in range(len(tsteps)-1):
        jp, jr, npq = jpq(jpqlist[t], ilist[t], Rlist[t])
        Rlist[t+1] = Rlist[t] + jr*dt
        jpqlist[t+1] = jp
        npqlist[t] = npq
    plt.plot(tsteps, jpqlist, label='Jpq')
    #plt.plot(tsteps, Rlist, label = 'RuBP')
    plt.plot(tsteps, ilist, label='Irradiance')
    npqlist[-1] = npqlist[-2]
    plt.plot(tsteps, npqlist, label='NPQ')
    plt.legend()
    plt.xlabel('time')
    plt.show()
        

interactive(children=(FloatSlider(value=5.0, description='pqo', max=10.0), IntSlider(value=250, description='R…